# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [1]:

%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient

import warnings
warnings.filterwarnings('ignore')

In [3]:
import re


In [4]:
import folium

In [5]:
import pandas as pd

In [6]:
# conexion con Mongo

cliente=MongoClient()

db=cliente.ufolab

In [31]:
db=cliente.ufolab
colec=db.ufolab

In [32]:
df=pd.DataFrame(list(colec.find({'country': 'us'},{'_id': 0, 'city': 1,'datetime':1,"latitude":1,"longitude":1,"distance":1})))

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63399 entries, 0 to 63398
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   63399 non-null  object
 1   city       63399 non-null  object
 2   latitude   63399 non-null  object
 3   longitude  63399 non-null  object
 4   distance   63399 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [34]:
df.datetime=df.datetime.str.extract(r'(\d{4})')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63399 entries, 0 to 63398
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   63399 non-null  object
 1   city       63399 non-null  object
 2   latitude   63399 non-null  object
 3   longitude  63399 non-null  object
 4   distance   63399 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [36]:
df.datetime=df.datetime.astype(dtype="int16")


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63399 entries, 0 to 63398
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   63399 non-null  int16 
 1   city       63399 non-null  object
 2   latitude   63399 non-null  object
 3   longitude  63399 non-null  object
 4   distance   63399 non-null  object
dtypes: int16(1), object(4)
memory usage: 2.1+ MB


In [38]:
df=df[df.datetime>=2004]

In [39]:
df.distance=df.distance.astype(dtype="float16")

In [45]:
df=df[(df.distance>300)&(df.distance<305)]

In [46]:
df

,datetime,city,latitude,longitude,distance
8581,2010,oxford,33.6141667,-85.835,302.25
14569,2012,homewood,33.4716667,-86.8008333,301.50
18740,2008,tazewell,36.4541667,-83.5694444,302.75
35134,2012,olney,38.7308333,-88.0852778,300.50
35736,2012,olney,38.7308333,-88.0852778,300.50
36204,2012,oxford,33.6141667,-85.835,302.25
50499,2013,manchester,37.1536111,-83.7619444,302.50
51965,2011,tazewell,36.4541667,-83.5694444,302.75
53451,2005,hueytown,33.4511111,-86.9966667,303.50


In [17]:
#insertar en mongo

In [47]:
colec.geoloc.insert_many(df.to_dict('records'))

In [48]:
# crear indice 2D

colec.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [56]:
df.reset_index(drop=True,inplace=True)

In [57]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], popup=str(df.city[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    
    return mapa

In [58]:
mapa=geomap(df.loc[:1500])
mapa

# La ciudad en Nashvile!!!!!!!!